In [ ]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import sympy
from scipy.linalg import expm
import scipy.constants as sc
from tqdm import tqdm_notebook
from scipy.fft import fft,ifft,fftfreq, dct, idct

In [ ]:
class lindbladian_Louville_operator():
    def __init__(self,N,Nt,h,J,b,t,tau,gamma,lamda):
        for local in locals().copy(): 
            ## equivalent of writing self.gamma = gamma,self.b = b etc.
            ## without having to rewrite it.
            exec('self.' + local +  ' = locals()[local]')
        
        ###initialising variables so they don't have to called in 
        ###each function
        self.initialise_variables()
        

        def initialise_variables(self,):
            #def H_s(self,N,h,J,b,t,tau):
            ####gamma is an array of 5 constants that control the dissipation
            si = qeye(2) #identity for a spin-1/2 particle
            sz = sigmaz()
            sx = sigmax()
            sigp = sigmap() ##plus ladder operator for spin 1/2
            sigm = sigmam() ##minus ladder operator for spin 1/2
            sz_list = []
            sx_list = []
            sigp_list = []
            sigm_list = []
            h_arr = h_rand(h,N)

            for n in range(N):
                op_list = []
                for m in range(N):
                    op_list.append(si) ##creates a list of identity operators for each particle

                op_list[n] = sz
                sz_list.append(tensor(op_list))

                op_list[n] = sx
                sx_list.append(tensor(op_list))
                ###could get greater efficiency by just calculating the plus/minus 
                ### at the boundaries where they're needed
                if n==0:
                    op_list[n] = sigp
                    sigp_list.append(tensor(op_list))

                    op_list[n] = sigm
                    sigm_list.append(tensor(op_list))
                if n==N-1:
                    op_list[n] = sigp
                    sigp_list.append(tensor(op_list))

                    op_list[n] = sigm
                    sigm_list.append(tensor(op_list))

            # construct the hamiltonian 
            H_S = 0
            A = 0 ##bulk dephasing and amplitude damping at the boundaries

            # energy splitting terms
            for n in range(N):
                H_S += h_arr[n] * sz_list[n]

                ##bulk dephasing
                #A += np.sqrt(gamma[0]) * sz_list[n]

            ##amplitude damping
           # A += gamma[1]*sigp_list[0]+gamma[2]*sigp_list[1] + gamma[3]*sigm_list[0] + gamma[4]*sigm_list[1]

                ##boundary condition for ends of the chain?
            # interaction terms
            for n in range(N-1): 
                H_S += J * sz_list[n] * sz_list[n+1]

            H1 =H_S
            ##kicking term
            Hk = 0
            for n in range(N):
                    Hk += b * sx_list[n]

            ##if t=tau/2, then this introduces the kicking term
            if (np.round(2*t/tau,3))==1:
                H_S += Hk

            ###Floquet operator
            U_T = (-1j*Hk).expm()*(-1j*H1).expm()

            gamma_p_w = lamda**2 * B
            
            t_list = np.linspace(0,1,Nt)
            U_t_list = []

            for i in range(Nt):
                ##don't have internet, find out how to actually write
                ##these two if statesments
                #if t[i] is less than tau/2:
                    out = (-1j*Hk*t[i]).expm()
                #if t[i] is more than tau/2:
                    out = ((-1j*Hk*t[i]).expm()*(-1j*H1).expm())[:] 

                U_t_list.append(out)
            
            floq_evecs = U_T.eigenstates()[1]
            floq_evals = U_T.eigenstates()[0]
            quasi_energies = -sc.hbar*np.angle(floq_evals)

            
            ##dimension of eigenvectors
            m = np.size(floq_evecs[0]) 

            ##setting up for a 3D array of dimensions:
            #dimension of system hilbert space x number of eigenvectors x number of time points
            mode_array = np.zeros((Nt,m,m)) 

            ##creates an array of dimension (Nt,m,m)
            U_t_matrix = np.rollaxis(np.dstack(U_t),-1) 

            ## creates an array of shape (m,m) where the second index refers to alpha
            floq_evecs_arr = np.dstack(floq_evecs)[:,0,:]

            ##creates an (Nt,m,m) array where the last index gives alpha and the second index is 
            ###the dimension of the basis vectors
            mode_array = np.matmul(U_t_matrix,floq_evecs_arr)

            ###this makes sure t=0 one is correct, but shouldnt be needed 
            ###if we move the kicking term to the middle of the time period
            mode_array[0,:,:] = floq_evecs_arr


            ###performs a fourier transform along the time (0) axis,
            ### creating an array of dimensions (Nt,m,m) where the first axis
            ###denotes q, the 3rd axis denotes alpha, and the second axis gives the
            ###vector.
            f_mode_array = fft(mode_array,axis=0)
            
            
        


        # def L(self,H_S,H_LS,A_pw,gamma_p_w):
        #     ###do sums


        #     si = qeye(2)
        #     op_list = []
        #     for m in range(N):
        #             op_list.append(si) ##creates a list of identitiy operators for each particle
        #     op_list = tensor(op_list)
        #     L = -1j*tensor(H,op_list) +1j*tensor(op_list,H.trans())
        #     return L 

        #'''''''''''''''''Level Statistics functions''''''''''''''''''''''
        def z_k(self,E_array):
            z_k = [] ##setting up ratio array

            for i in range(np.size(E_array)):
                #_sorted_i= np.sort(E_array-E_array[i])
                E_idx_sorted = np.argsort(np.abs(E_array-E_array[i]))
                NN = E_array[E_idx_sorted[1]]
                NNN = E_array[E_idx_sorted[2]]

                z_k.append((NN-E_array[i])/(NNN-E_array[i]))
            z_k = np.asarray(z_k) 
            return z_k
        def h_rand(self,h,N):
            out = np.random.uniform(-h,h,N)
            return out

        def pdf_val(self,value,ratios,width):
            ##sorting in terms of size(only works for real)
            srtd_rts = np.sort(ratios)

            ##finding the indices of the values that lie at 
            ##the top and bottom of the small section centred on the value
            upper_bound = np.argmin(np.abs(srtd_rts-value-width))

            lower_bound = np.argmin(np.abs(srtd_rts-value+width))

            rho = ((upper_bound-lower_bound)/np.size(ratios))/(2*width)
            ###rho gives the number of values in the range (value-width) to (value+width)
            ##divided by the range
            return rho
        def pdf(self,ratios,nv):
            pdf_list = []
            delta = (np.max(ratios) - np.min(ratios))/nv
            width = delta/2
            values = np.linspace(np.min(ratios),np.max(ratios),nv)
            for i in range(nv):
                pdf_val_i = pdf_val(values[i],ratios,width)
                #print(pdf_val_i)
                pdf_list.append(pdf_val_i)

            pdf = np.asarray(pdf_list)
            return pdf,values    






        """"""""""""""""""'Functions for diagonalisation and setting up solution'""""""""""""""""""""

        
        def Apw(self,p,w):
            ##creating q,q' lists

            ##q_dash_list = (0,1,2,...,N-1-p)
            q_dash_list = np.linspace(0,N-1-p,N-p)
             ##q_list = (p,p+1,p+2,...,N-1)
            q_dash_list = q_list - p 

            ##creating e_a, e_b lists (quasienergy lists),
            ##quasienergy list may have to be turned into array for indexing etc

            #upper limit is quasienergies[-1]-w
            upper_limit = quasi_energies[-1]-w
            upper_limit_index = np.argmin(quasi_energies-upper_limit)
            #e_a = quasi_energies[:upper_limit_index]
            e_a_list = []
            e_a_index_list = []
            e_b_list = []
            e_b_index_list = []
            ##iterates from 0 through every quasi energy until the maximum-w (maximum-w = max poss e_a)
            for i in range(upper_limit_index): 
                poss_e_a = quasi_energies[i]
                poss_e_b = poss_e_b+w

                ##if statement checks to see if poss_e_b is a quasi energy up to 3 decimal places

                if np.min(np.round(np.abs(quasi_energies-poss_e_b),3)) ==0:
                    e_a_list.append(poss_e_a)
                    e_a_index_list.append(i)
                    e_b_list.append(poss_e_b)
                    e_b_index_list.append(np.argmin(np.round(np.abs(quasi_energies-poss_e_b),3)))

            ###the following is a for loop method to evaluate A_pw, its clearly
            ### very inefficient, I should work out an array method that doesnt need for loops

            A_pw = 0
            for i in range(np.size(e_a_list)):
                for j in range(np.size(q_list)):

                    A_pw += 0#### matrix element of A operator 
                    ##multiplied by outer product of the floquet states
            return A_pw

        def A_pw_list(self,):
            ##finding list of possible w frequencies
            ## DO THIS
            #w_list = 
            ##finding list of possible p frequencies
            p_list = np.linspace(0,N-1,N)

            for i in range(np.size(w_list)):
                for j in range(np.size(p_list)):
                    A_pw_list.append(A_pw(A,p_list[j],w_list[i],f_mode_array,quasi_energies,N))

            return A_pw_list









In [ ]:

##setting parameters
N = 10
h = 10
J = 1
b = 1
t = 1
tau = 1
H = H_s(N,h,J,b,t,tau)[0]
U = H_s(N,h,J,b,t,tau)[1]
H1 = H_s(N,h,J,b,t,tau)[2]
Hk = H_s(N,h,J,b,t,tau)[3]
E_array = H.eigenenergies()